In [3]:
from corpus_processor.src.corpus_to_text import read_documents_from_bucket, read_txt_files_from_bucket
from corpus_processor.src.file_processor import document_processor
from corpus_processor.src.corpus_to_text import read_txt_files_from_bucket
from context_generator.src.split_docs import split_docus_to_chuncks
from context_generator.src.zero_shot_classifier import zero_shot_classification

from config import config

# document_processor()
# read_documents_from_bucket()
docs = read_txt_files_from_bucket()
chunks = split_docus_to_chuncks(docs)
if config.do_zero_shot_classification:
    topics = ["challenge", "innovation", "investment", "achievement"]
    result = zero_shot_classification(chunks, topics, 0.8)


txt/
txt/AI_for_Health_CI
txt/JohnsonJohnson2022-annual-report
txt/Pfizer_Annual_Review
txt/SANOFI-Integrated-Annual-Report-2022-EN
txt/bayer_future_trends
txt/emerging from disruption the future of pharma operations strategy
txt/entire-merck-ar22
txt/pwc_next_in_pharmaceuticals
txt/pwc_pharma_2023
txt/roche_Annual_Report_2022
txt/the-economic-potential-of-generative-ai-the-next-productivity-frontier-vf
# of chuncks from all the documents = 3021


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [4]:
import pickle
dict_poi = pickle.load(open('chunks_zero_shot_classification.p','rb'))
print(dict_poi.keys())
dict_poi['innovation']

[('*PARAGRAPH_INDICATOR*Al for Health Capgemini invent\n*PARAGRAPH_INDICATOR*Ⓡ\n*PARAGRAPH_INDICATOR*HEALTH & Al\n*PARAGRAPH_INDICATOR*NOW AND NEXT\n*PARAGRAPH_INDICATOR*GE\n*PARAGRAPH_INDICATOR*2022 EDITION\n*PARAGRAPH_INDICATOR*Al for Health Capgemini invent\n*PARAGRAPH_INDICATOR*Ⓡ\n*PARAGRAPH_INDICATOR*Al for Health\n*PARAGRAPH_INDICATOR*Capgemini invent\n*PARAGRAPH_INDICATOR*HEALTH & Al\n*PARAGRAPH_INDICATOR*NOW AND NEXT\n*PARAGRAPH_INDICATOR*Health & Al: Now and Next 2022 edition 3\n*PARAGRAPH_INDICATOR*Foreword\n*PARAGRAPH_INDICATOR*Since 2016, tech leaders are developing their collaborations with\nhealthcare institutions: Google launched 12 projects, Microsoft 7\nand Amazon 5. More recently, COVID paved the way for a massive\ninterest in data and Al domain, stakeholders are now fully\nembracing it, learning and sometimes succeeding on their way.\n*PARAGRAPH_INDICATOR*In 2022, we are celebrating the 5th edition of the Al for Health Summit. 5-years',
  0.8419631719589233,
  0)]

In [ ]:
from context_generator.src.zero_shot_classifier import zero_shot_classification
topics = ["challenge", "innovation", "investment", "achievement", "best practice", "priority"]

a = zero_shot_classification(chunks[0:10], topics, 0.8)
print(a)

In [2]:
from typing import Sequence

from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore

# TODO(developer): Uncomment these variables before running the sample.
project_id = "gen-hi-france-genai-force1"
location = "us" # Format is "us" or "eu"
processor_id = "YOUR_PROCESSOR_ID" # Create processor before running sample
processor_version = "rc" # Refer to https://cloud.google.com/document-ai/docs/manage-processor-versions for more information
file_path = "gs://corpus-fr1-hack/pdf"
mime_type = "application/pdf" # Refer to https://cloud.google.com/document-ai/docs/file-types for supported file types

def process_document_ocr_sample(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
) -> None:
    # Online processing request to Document AI
    document = process_document(
        project_id, location, processor_id, processor_version, file_path, mime_type
    )

    text = document.text
    print(f"Full document text: {text}\n")
    print(f"There are {len(document.pages)} page(s) in this document.\n")

    for page in document.pages:
        print(f"Page {page.page_number}:")
        # print_page_dimensions(page.dimension)
        # print_detected_langauges(page.detected_languages)
        print_paragraphs(page.paragraphs, text)
        # print_blocks(page.blocks, text)
        # print_lines(page.lines, text)
        # print_tokens(page.tokens, text)

        # Currently supported in version `pretrained-ocr-v1.1-2022-09-12`
        if page.image_quality_scores:
            print_image_quality_scores(page.image_quality_scores)


def print_page_dimensions(dimension: documentai.Document.Page.Dimension) -> None:
    print(f"    Width: {str(dimension.width)}")
    print(f"    Height: {str(dimension.height)}")


def print_detected_langauges(
    detected_languages: Sequence[documentai.Document.Page.DetectedLanguage],
) -> None:
    print("    Detected languages:")
    for lang in detected_languages:
        code = lang.language_code
        print(f"        {code} ({lang.confidence:.1%} confidence)")


def print_paragraphs(
    paragraphs: Sequence[documentai.Document.Page.Paragraph], text: str
) -> None:
    print(f"    {len(paragraphs)} paragraphs detected:")
    for paragraph in paragraphs:
        first_paragraph_text = layout_to_text(paragraph.layout, text)
        print(f"        First paragraph text: {repr(first_paragraph_text)}")
        # last_paragraph_text = layout_to_text(paragraphs[-1].layout, text)
        # print(f"        Last paragraph text: {repr(last_paragraph_text)}")


def print_blocks(blocks: Sequence[documentai.Document.Page.Block], text: str) -> None:
    print(f"    {len(blocks)} blocks detected:")
    first_block_text = layout_to_text(blocks[0].layout, text)
    print(f"        First text block: {repr(first_block_text)}")
    last_block_text = layout_to_text(blocks[-1].layout, text)
    print(f"        Last text block: {repr(last_block_text)}")


def print_lines(lines: Sequence[documentai.Document.Page.Line], text: str) -> None:
    print(f"    {len(lines)} lines detected:")
    first_line_text = layout_to_text(lines[0].layout, text)
    print(f"        First line text: {repr(first_line_text)}")
    last_line_text = layout_to_text(lines[-1].layout, text)
    print(f"        Last line text: {repr(last_line_text)}")


def print_tokens(tokens: Sequence[documentai.Document.Page.Token], text: str) -> None:
    print(f"    {len(tokens)} tokens detected:")
    first_token_text = layout_to_text(tokens[0].layout, text)
    first_token_break_type = tokens[0].detected_break.type_.name
    print(f"        First token text: {repr(first_token_text)}")
    print(f"        First token break type: {repr(first_token_break_type)}")
    last_token_text = layout_to_text(tokens[-1].layout, text)
    last_token_break_type = tokens[-1].detected_break.type_.name
    print(f"        Last token text: {repr(last_token_text)}")
    print(f"        Last token break type: {repr(last_token_break_type)}")


def print_image_quality_scores(
    image_quality_scores: documentai.Document.Page.ImageQualityScores,
) -> None:
    print(f"    Quality score: {image_quality_scores.quality_score:.1%}")
    print("    Detected defects:")

    for detected_defect in image_quality_scores.detected_defects:
        print(f"        {detected_defect.type_}: {detected_defect.confidence:.1%}")


def process_document(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    # You must set the `api_endpoint` if you use a location other than "us".
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, processor_version
    )

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type=mime_type),
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document


def layout_to_text(layout: documentai.Document.Page.Layout, text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in layout.text_anchor.text_segments:
        start_index = int(segment.start_index)
        end_index = int(segment.end_index)
        response += text[start_index:end_index]
    return response


In [8]:
project_id = "gen-hi-france-genai-force1"
location = "us" # Format is "us" or "eu"
processor_id = "66868ad1d8c4e090" # Create processor before running sample
processor_version = "pretrained-ocr-v1.0-2020-09-23" # Refer to https://cloud.google.com/document-ai/docs/manage-processor-versions for more information
file_path = r'C:\Users\mmozafar\OneDrive - Capgemini\Projects\GenAI_Hackathon\pdf-convertor\input\test_1.pdf'
mime_type = "application/pdf"
process_document_ocr_sample(project_id=project_id, location=location, processor_id=processor_id, processor_version=processor_version, file_path=file_path, mime_type=mime_type)

Full document text: Al for Health Capgemini invent
HEALTH & Al
NOW AND NEXT
2022 EDITION
Al for Health Capgemini invent
|
R
Al for Health Capgemini invent
HEALTH & Al
NOW AND NEXT
Health & Al: Now and Next | 2022 edition 3
Foreword
Since 2016, tech leaders are developing their collaborations with
healthcare institutions: Google launched 12 projects, Microsoft 7
and Amazon 5. More recently, COVID paved the way for a massive
interest in data and Al domain, stakeholders are now fully
embracing it, learning and sometimes succeeding on their way.
In 2022, we are celebrating the 5th edition of the Al for Health Summit. 5-years
is quite young, but old enough to see changes in this fast-evolving industry
that is catching up at such a speedy pace.
After the success of the two first editions, we come back this year even stronger
to report on where healthcare organizations stand in their data and Al
journey. We have extended our scope and we now cover Pharmas, MedTechs,
Tech providers, Startups, 

In [1]:
from corpus_processor.src.corpus_to_text import read_documents_from_local

data_dir = r'C:\Users\mmozafar\OneDrive - Capgemini\Projects\GenAI_Hackathon\pdf-convertor\input'
data = read_documents_from_local(data_dir, read_from='pdf')

print(data)

The PDF <_io.BufferedReader name='C:\\Users\\mmozafar\\OneDrive - Capgemini\\Projects\\GenAI_Hackathon\\pdf-convertor\\input\\entire-merck-ar22.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


# of pages in .pdf documents: 11
[Document(page_content='\x0c\x0c®\n\nHEALTH & AI\nNOW AND NEXT\n\nHealth & AI: Now and Next    2022 edition\n\n3\n\n\x0cForeword\n\nSince 2016, tech leaders are developing their collaborations with \n\nhealthcare institutions: Google launched 12 projects, Microsoft 7 \n\nand Amazon 5. More recently, COVID paved the way for a massive \n\ninterest  in  data  and  AI  domain,  stakeholders  are  now  fully \n\nembracing it, learning and sometimes succeeding on their way.\n\nIn 2022, we are celebrating the 5th edition of the AI for Health Summit. 5-years \nis quite young, but old enough to see changes in this fast-evolving industry \nthat is catching up at such a speedy pace.\n\nAfter the success of the two first editions, we come back this year even stronger \nto  report  on  where  healthcare  organizations  stand  in  their  data  and  AI \njourney. We have extended our scope and we now cover Pharmas, MedTechs, \nTech  providers,  Startups,  Hospitals,  

In [8]:
print(data[0].metadata)

{'source': 'C:\\Users\\mmozafar\\OneDrive - Capgemini\\Projects\\GenAI_Hackathon\\pdf-convertor\\input\\AI_for_Health_CI.pdf'}


In [11]:
data[0].page_content

'\x0c\x0c®\n\nHEALTH & AI\nNOW AND NEXT\n\nHealth & AI: Now and Next    2022 edition\n\n3\n\n\x0cForeword\n\nSince 2016, tech leaders are developing their collaborations with \n\nhealthcare institutions: Google launched 12 projects, Microsoft 7 \n\nand Amazon 5. More recently, COVID paved the way for a massive \n\ninterest  in  data  and  AI  domain,  stakeholders  are  now  fully \n\nembracing it, learning and sometimes succeeding on their way.\n\nIn 2022, we are celebrating the 5th edition of the AI for Health Summit. 5-years \nis quite young, but old enough to see changes in this fast-evolving industry \nthat is catching up at such a speedy pace.\n\nAfter the success of the two first editions, we come back this year even stronger \nto  report  on  where  healthcare  organizations  stand  in  their  data  and  AI \njourney. We have extended our scope and we now cover Pharmas, MedTechs, \nTech  providers,  Startups,  Hospitals,  Research  labs,  Insurance,  Patients \nand Regulators. 

In [10]:
import re
re.split(pattern="\n\n", string=data[0].page_content)


['\x0c\x0c®',
 'HEALTH & AI\nNOW AND NEXT',
 'Health & AI: Now and Next    2022 edition',
 '3',
 '\x0cForeword',
 'Since 2016, tech leaders are developing their collaborations with ',
 'healthcare institutions: Google launched 12 projects, Microsoft 7 ',
 'and Amazon 5. More recently, COVID paved the way for a massive ',
 'interest  in  data  and  AI  domain,  stakeholders  are  now  fully ',
 'embracing it, learning and sometimes succeeding on their way.',
 'In 2022, we are celebrating the 5th edition of the AI for Health Summit. 5-years \nis quite young, but old enough to see changes in this fast-evolving industry \nthat is catching up at such a speedy pace.',
 'After the success of the two first editions, we come back this year even stronger \nto  report  on  where  healthcare  organizations  stand  in  their  data  and  AI \njourney. We have extended our scope and we now cover Pharmas, MedTechs, \nTech  providers,  Startups,  Hospitals,  Research  labs,  Insurance,  Patients \nand 

In [84]:
i=3
print(data[i].metadata)
data[i].page_content

{'source': 'C:\\Users\\mmozafar\\OneDrive - Capgemini\\Projects\\GenAI_Hackathon\\pdf-convertor\\input\\AI_for_Health_CI.pdf', 'page': 3}


'4 Health & AI: Now and Next  \n   2022 editionForeword\nIn 2022, we are celebrating the 5th edition of the AI for Health Summit. 5-years \nis quite young, but old enough to see changes in this fast-evolving industry  \nthat is catching up at such a speedy pace.\nAfter the success of the two first editions, we come back this year even stronger \nto report on where healthcare organizations stand in their data and AI \njourney. We have extended our scope and we now cover Pharmas, MedTechs, \nTech providers, Startups, Hospitals, Research labs, Insurance, Patients  \nand Regulators. \nAs part of the “AI for Health Global Conference”, held on November 16, 2022, \nthis report gathers a large panel of 140+ respondents into an online survey, \nalong with 22 deep-dive interviews with field change makers that bring \ntheir expertise in the domain. We have reached some statistical significance \nand international stature. We would like to thank our panel for their precious \ninsights.\nSimilar to

In [ ]:
from langchain.document_loaders import PyPDFLoader
file_path = r'C:\Users\mmozafar\OneDrive - Capgemini\Projects\GenAI_Hackathon\pdf-convertor\input\SANOFI-Integrated-Annual-Report-2022-EN.pdf'
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()


In [34]:
from langchain.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader(file_path, mode="elements")

data = loader.load()



'possible'

In [59]:
from langchain.document_loaders import PyPDFium2Loader

loader = PyPDFium2Loader(file_path)

data = loader.load()

In [ ]:
print(data[2].page_content)

In [65]:
from langchain.document_loaders import PDFMinerLoader

loader = PDFMinerLoader(file_path)

data = loader.load()

In [68]:
data[0].page_content

'possible\n\nOur Integrated Annual Report 2022\n\n\x0cIntegrated Annual Report 2022\n\n02\n\nIn this report\n\n03 -\n\nIntroduction \n• Foreword by Paul Hudson \n• Our Play to Win Strategy\n• How We’re Turning the Impossible \ninto Possible \n\n11 -\n\nChasing the Miracles of Science\n\n• A Pipeline of Possibilities\n• Our Scientific Progress in 2022\n• Connecting the Dots with Digital \n• Revving up Manufacturing and Supply \n• Making Clinical Trials More Inclusive \n\n24 -\n\nImproving People’s Lives \n• Making a Lasting Impact on Society \n• A Think & Do Tank to Reimagine Philanthropy \n• Protecting our Planet throughout the\nProduct Life Cycle \n• “All In” for Diversity, Equity, and Inclusion \n\n38 -\n\nHow We Create Shared Value \n• Corporate Governance\n• Financial Performance \n• Creating Value for All \n\nAnais Brafine, \nAssociate Scientist, Belgium \n\n\x0cIntegrated Annual Report 2022\n\n03\n\nForeword by Paul Hudson\n\nWhile the past few years have carried with them a \nmu